# Лабораторная работа 6. 
# Прогнозирование стоимости акций с помощью линейной регрессии

В данной работе вам необходимо построить регрессионную модель для прогноза стоимости акций 
- для дат, находящихся в рассматриваемом временном периоде
- для дат после некоторого временного периода (т.е. на будущее)

Для построения модели линейной регрессии и оценки ее качества вы будете использовать инструменты из пакета `scikit-learn` -  пользуйтесь руководством пользователя https://scikit-learn.org/stable/user_guide.html  для этого пакета.

В датасете представлена информация о торгах по акциям британской кинокомпании Seven Arts Entertainment Inc:
- Дата торгов (Date)
- Цена открытия (Open)
- Цена закрытия (Close)
- Наибольшая цена (High)
- Наименьшая цена (Low)
- Торговый объем (Traded Volume)
- Оборот (Turnover)

Необходимо постороить регрессию цены закрытия на время.

## Загрузка данных

### Импорт библиотек

In [41]:
import numpy as np
import pandas as pd
import datetime

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
plt.rc('figure', figsize=(16,10))
plt.rc('lines', markersize=4)

📌📌📌 **Задание 1**

Загрузите таблицу данных из файла (`sap_stock_1.csv` - для варианта 1,`sap_stock_4.csv` - для варианта 2,`sap_stock_4.csv` - для варианта 3,`sap_stock_4.csv` - для варианта 4).

Выведите на экран несколько первых строк датасета.
С помощью `pd.info()` и `pd.describe()` выведите информацию о структуре датасета и описательные статистики для каждой из переменных

In [2]:
# Ваш код

📌📌📌 **Задание 2**

Сформируйте датасет `df`, по которому будет строиться и оцениваться модель регрессии.
Датасет должен содержать:
- столбец значений переменной-отклика `Сlose`, 
- столбец значений переменной `Date` (который мы не будем включать в модель, а используем только для визуализации)

Индексы датасета - номера моментов времени. Их вы будете использовать в качестве единственного предиктора. 

In [42]:
# Ваш код

## Предварительный визуальный анализ данных

Построим график зависимости *цены акции* от *времени*. Убедимся, что несмотря на многочисленные колебания цены имеетя тренд на рост цены. Возможно, этот тренд можно приблизить линейной функцией.

In [9]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [ ]:
# Подключение модуля matplotlib для графиков с датами
import matplotlib.dates as mdates

years = mdates.YearLocator() # Извлекать из даты будем только год
yearsFmt = mdates.DateFormatter('%Y') # Формат представления года

# Create subplots to plot graph and control axes
fig, ax = plt.subplots()
ax.plot(df['Date'], df['Close'])

# Format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)

# Set figure title
plt.title('История цен закрытия акций', fontsize=16)
# Set x label
plt.xlabel('Дата', fontsize=14)
# Set y label
plt.ylabel('Цена акции, $', fontsize=14)

# Rotate and align the x labels
fig.autofmt_xdate()

# Show plot
plt.show()

## Обучение модели парной линейной регрессии
 
Наша цель - обучение модели линейной регрессии вида
$$
Y = \beta_0 + \beta_1 X
$$

где   
* $Y$  - отклик (цена акции)
* $X$  - независимая переменная (момент времени)
* $\beta_0$, $\beta_1$ коэффициенты регрессии

Обучение регрессии состоит в подборе таких $\beta_0$ и $\beta_1$ , чтобы  **Сумма квадратов отклонений (RSS)** была минимальна.

📌📌📌 **Задание 3**

Разделите датасет `df` на тренировочный (`train`) и тестовый (`test`) в соотношении 80:20.

В датасет `X_train` отделите информацию о значениях предиктора  *момент времени* из датасета `train`(он находится в индексе), в 
`y_train` - значения отклика *цена* из датасета `train`.

Аналогично создайте `X_test` и `y_test`.

Обучите линейную регрессию на `(X_train, y_train)`

Вычислите значение коэффициента детерминации полученной модели на тренировочном датасете.

Хорошо ли линейная регрессия объясняет дисперсию отклика на обучающей выборке?

In [11]:
# импорт модуля для случайного разделения выборок 
from sklearn.model_selection import train_test_split

In [12]:
# Ваш код - разделение выборки

In [14]:
# импорт линейного регрессора из модуля, содержащего различные линейные модели
from sklearn.linear_model import LinearRegression

In [15]:
# Ваш код - обучение линейной модели

LinearRegression()

In [ ]:
#Вычисляем R^2 на обучающей выборке
model.score(X_train, y_train)

### Изобразим линию регрессии

In [ ]:
plt.figure(1, figsize=(16,10))
plt.title('Linear Regression | Price vs Time')
plt.scatter(X_train, y_train, edgecolor='w', label='Actual Price')
plt.plot(np.array(X_train), model.predict(X_train), color='r', label='Predicted Price')
plt.xlabel('Integer Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

## Оценка модели регрессии
**Метрики ошибки:**

   **Mean Squared Error (MSE)** - среднеквадратичная ошибка:
    $$
    \frac{1}{N} \sum_{i = 1}^{N} (y_i - \hat{y}_i)^2
    $$

   **Root Mean Squared Error (RMSE)** - корень из среднеквадратичной ошибки:
    $$
    \sqrt{\frac{1}{N} \sum_{i = 1}^{N} (y_i - \hat{y}_i)^2}
    $$

Это те метрики, которые необходимо минимизировать.

**Метрики качества:**

   **Coefficient of determination** - коэффициент детерминации (измеряет, насколько хорошо общая дисперсия отклика объясняется моделью регрессии):
    $$
    R^2 = 1 - \frac{RSS}{TSS}
    $$

   где

   **Residual Sum of Squares (RSS)** - сумма квадратов регрессионных остатков
    $$
    RSS = \sum_{i = 1}^{N} \epsilon_i^2 = \sum_{i = 1}^{N} (y_i - \hat{y}_i)^2
    $$

   **Total Sum of Squares (TSS)**  - общая сумма квадратов отклонений от среднего значения
    $$
    TSS = \sum_{i = 1}^{N} (y_i - \bar{y}_i)^2
    $$

$R^2$ - метрика, которую необходимо максимизировать.

📌📌📌 **Задание 4**

С помощью полученной вами модели регрессии постройте прогноз на тестовой выборке (используйте функцию `LinearRegression.predict()`).


На тестовой выборке вычислите значения метрик **MSE**, **RMSE**, **$R^2$** (используйте функции  `metrics.mean_squared_error()` и `metrics.r2_score()` )

По аналогии с предыдущим заданием, изобразите линию регрессии вместе с точками данных из тестовой выборки.

Как вы оцениваете качество регрессионной модели на тестовой выборке?

In [19]:
# импортируем из sklearn модуль, содержащие метрики качества
from sklearn import metrics

In [44]:
# Ваш код

📌📌📌 **Задание 5**

Теперь обучите линейную регрессионную модель и оцените ее качество, по-другому разбив датасет на тренировочный и тестовый.
Разбивать необходимо также в соотношении 80:20, но не случайным образом, а поместив в обучающую выборку первые 80% объектов, а в тестовую - последние 20%.

В ячейках ниже напишите код, который:
1. Разбивает датасет `df` на новые тренировочный и тестовый датасеты.
2. Обучает модель регрессии и на тренировочном датасете вычисляет значение $R^2$
3. Строит прогноз на тестовом датасете и вычисляет на нем метрики **MSE**, **RMSE**, **$R^2$**
4. На одном графике отображает объекты тренировочного и тестового датасета и линию регрессии.

Как вам качество регрессии: стало лучше или хуже по сравнению со случаем случайного разбиения на тренировочный и тестовый датасеты? Как думаете, почему?

In [22]:
# Ваш код

## Квазилинейная регрессия

В случае, если линейная модель плохо объясняет дисперсию отклика, можно использовать обощение линейной модели - квазилинейную регрессию, в которую предикторы входят нелинейным образом.

Например, если в $f(x)$ кроме тренда присутствуют циклические колебания значений отклика, можно попробовать использовать в качестве предикторов не только $x$, но и элементы тригонометрического ряда Фурье функции $f(x)$: $$\sin\left(\frac{2 n\pi}{w} \cdot x\right) \text{ и } \cos\left(\frac{2 n\pi }{w} \cdot x\right),$$
гдe $w$ - период колебаний, значения $n=1,2,3...w$ 

📌📌📌 **Задание 6 (только для варианта 4)**

В датасет `df` добавьте столбцы со значениями предикторов, полученных как элементы ряда Фурье, где переменная $x$ - момент времени из `df.index`, период колебаний $w$ подберите по графикам выше. 

Число элементов ряда Фурье выберите самостоятельно (начните с $n=\{1, 2\}$, затем увеличивайте число элементов, пока качество регрессии продолжает существенно улучшаться)

Повторите работу, проведенную вами в задании 3 и задании 4.

Сравните качество линейной и квазилинейной регрессии. Удалось ли улучшить качество линейной модели добавлением нелинейных предикторов? 


In [29]:
t=df.index
df['time']=t
w=... # запишите подобранный по графику период колебаний

for i in range(1,...):# вместо ... вставьте свое число элементов ряда Фурье
    sin="s"+str(i)
    cos="c"+str(i)
    df[sin]=np.sin(2*i*np.pi*t/w)
    df[cos]=np.cos(2*i*np.pi*t/w)
df.head()

In [ ]:
# Ваш код